In [3]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String
from sqlalchemy.exc import NoSuchTableError
# import support_functions
from time import time
import datetime

def create_engine_with_db(database_url):
    return create_engine(database_url)

def reflect_table(engine, table_name):
    meta = MetaData()
    try:
        return Table(table_name, meta, autoload_with=engine)
    except NoSuchTableError:
        return None

def update_table_structure(engine, table, dataframe):
    existing_columns = set(table.columns.keys())
    new_columns = set(dataframe.columns) - existing_columns

    with engine.connect() as conn:
        for column in new_columns:
            # Add new columns to the table, assuming string type for simplicity.
            # Modify as needed for different data types.
            new_column = Column(column, String)
            new_column.create(table, connection=conn)

def data_exists(engine, table_name, year):
    table = reflect_table(engine, table_name)
    if not table:
        return False

    with engine.connect() as conn:
        query = select([table]).where(table.c['UTC Time at End of Hour'].like(f'%{year}%'))
        result = conn.execute(query).fetchall()
        return len(result) > 0

def download_data(url):
    return pd.read_csv(url, low_memory=False)

def process_and_load_data(years, url_template, table_name, engine):
    table = reflect_table(engine, table_name)
    if not table:
        print(f"Table '{table_name}' does not exist.")
        return

    for year in years:
        if not data_exists(engine, table_name, year):
            for half in ['Jan_Jun', 'Jul_Dec']:
                url = url_template.format(year=year, half=half)
                dataframe = download_data(url)
                update_table_structure(engine, table, dataframe)
                dataframe.to_sql(table_name, con=engine, index=False, if_exists='append')
                print(f'Data from {url} loaded into {table_name}.')

def main():
    # log_file = support_functions.log_output("outputs/download_logs/eia930/")
    try:
        t0 = time()

        database_url = 'sqlite:///C:/Users/adamh/OneDrive - Marquette University/Personal/Jobs/Health Catalyst/EIA930_database.db'
        engine = create_engine_with_db(database_url)

        years = [2018, 2019, 2020, 2021, 2022, 2023]
        url_template_balance = 'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_{year}_{half}.csv'
        url_template_subregion = 'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_SUBREGION_{year}_{half}.csv'

        process_and_load_data(years, url_template_balance, 'eia930_balance_raw', engine)
        process_and_load_data(years, url_template_subregion, 'eia930_subregion_raw', engine)

        t1 = time()
        print(str(datetime.timedelta(seconds=round(t1 - t0))))

    finally:
        print('finished')
        # log_file.close()


if __name__ == "__main__":
    main()


Table 'eia930_balance_raw' does not exist.
Table 'eia930_subregion_raw' does not exist.
0:00:00
finished


In [12]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, select, func, inspect
from sqlalchemy.exc import NoSuchTableError
# import support_functions
from time import time
import datetime

def create_engine_with_db(database_url):
    return create_engine(database_url)

def get_or_create_table(engine, table_name, dataframe):
    meta = MetaData()
    inspector = inspect(engine)

    if not inspector.has_table(table_name):
        # Create table if it doesn't exist
        columns = [Column(name, String) for name in dataframe.columns]
        table = Table(table_name, meta, *columns)
        meta.create_all(engine)
    else:
        table = Table(table_name, meta, autoload_with=engine)

    return table

def update_table_structure(engine, table, dataframe):
    meta = MetaData()
    meta.reflect(bind=engine)
    existing_columns = set(table.columns.keys())
    new_columns = set(dataframe.columns) - existing_columns

    with engine.connect() as conn:
        for column in new_columns:
            # SQLite requires columns to be quoted if they contain special characters or spaces
            quoted_column = f'"{column}"' if ' ' in column or any(c in column for c in '()[]{}<>-+=*%&^$#@!~') else column
            conn.execute(f'ALTER TABLE {table.name} ADD COLUMN {quoted_column} STRING')

def data_exists(engine, table_name, year):
    meta = MetaData(engine)
    table = Table(table_name, meta, autoload=True)

    query = select([func.count()]).select_from(table).where(table.c['Data Date'].like(f'%{year}%'))
    result = engine.execute(query).scalar()
    return result > 0

def download_data(url):
    try:
        return pd.read_csv(url, low_memory=False, on_bad_lines='skip')
    except pd.errors.ParserError as e:
        print(f"Parser error while reading {url}: {e}")
        return None

def process_and_load_data(years, url_template, table_name, engine):
    for year in years:
        for half in ['Jan_Jun', 'Jul_Dec']:
            url = url_template.format(year=year, half=half)
            dataframe = download_data(url)

            if dataframe is not None:
                table = get_or_create_table(engine, table_name, dataframe)
                update_table_structure(engine, table, dataframe)

                if not data_exists(engine, table_name, year):
                    dataframe.to_sql(table_name, con=engine, index=False, if_exists='append')
                    print(f'Data from {url} loaded into {table_name}.')
            else:
                print(f"Failed to download or parse data from {url}")

def main():
    # log_file = support_functions.log_output("outputs/download_logs/eia930/")
    try:
        t0 = time()

        database_url = 'sqlite:///C:/Users/adamh/OneDrive - Marquette University/Personal/Jobs/Health Catalyst/EIA930_database.db'
        engine = create_engine_with_db(database_url)

        years = [2018, 2019, 2020, 2021, 2022, 2023]
        url_template_balance = 'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_{year}_{half}.csv'
        url_template_subregion = 'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_SUBREGION_{year}_{half}.csv'

        process_and_load_data(years, url_template_balance, 'eia930_balance_raw', engine)
        process_and_load_data(years, url_template_subregion, 'eia930_subregion_raw', engine)

        t1 = time()
        print(str(datetime.timedelta(seconds=round(t1 - t0))))

    finally:
        print('shazam')
        # log_file.close()

if __name__ == "__main__":
    main()


shazam


KeyError: 'Data Date'

In [14]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, select, func, inspect
from sqlalchemy.exc import NoSuchTableError
# import support_functions
from time import time
import datetime

def create_engine_with_db(database_url):
    return create_engine(database_url)

def get_or_create_table(engine, table_name, dataframe):
    meta = MetaData()
    inspector = inspect(engine)

    if not inspector.has_table(table_name):
        # Create table if it doesn't exist
        columns = [Column(name, String) for name in dataframe.columns]
        table = Table(table_name, meta, *columns)
        meta.create_all(engine)
    else:
        table = Table(table_name, meta, autoload_with=engine)

    return table

def update_table_structure(engine, table, dataframe):
    meta = MetaData()
    meta.reflect(bind=engine)
    existing_columns = set(table.columns.keys())
    new_columns = set(dataframe.columns) - existing_columns

    with engine.connect() as conn:
        for column in new_columns:
            # SQLite requires columns to be quoted if they contain special characters or spaces
            quoted_column = f'"{column}"' if ' ' in column or any(c in column for c in '()[]{}<>-+=*%&^$#@!~') else column
            conn.execute(f'ALTER TABLE {table.name} ADD COLUMN {quoted_column} STRING')

def data_exists(engine, table_name, year, date_column):
    meta = MetaData(engine)
    table = Table(table_name, meta, autoload=True)

    if date_column not in table.c:
        print(f"Column '{date_column}' not found in the table '{table_name}'.")
        return False

    query = select([func.count()]).select_from(table).where(table.c[date_column].like(f'%{year}%'))
    result = engine.execute(query).scalar()
    return result > 0

def download_data(url):
    try:
        dataframe = pd.read_csv(url, low_memory=False, on_bad_lines='skip')
        print(f"Columns in {url}: {dataframe.columns.tolist()}")
        return dataframe
    except pd.errors.ParserError as e:
        print(f"Parser error while reading {url}: {e}")
        return None

def process_and_load_data(years, url_template, table_name, engine, date_column):
    for year in years:
        for half in ['Jan_Jun', 'Jul_Dec']:
            url = url_template.format(year=year, half=half)
            dataframe = download_data(url)

            if dataframe is not None:
                table = get_or_create_table(engine, table_name, dataframe)
                update_table_structure(engine, table, dataframe)

                if not data_exists(engine, table_name, year, date_column):
                    dataframe.to_sql(table_name, con=engine, index=False, if_exists='append')
                    print(f'Data from {url} loaded into {table_name}.')
            else:
                print(f"Failed to download or parse data from {url}")

def main():
    # log_file = support_functions.log_output("outputs/download_logs/eia930/")
    try:
        t0 = time()

        database_url = 'sqlite:///C:/Users/adamh/OneDrive - Marquette University/Personal/Jobs/Health Catalyst/EIA930_database.db'
        engine = create_engine_with_db(database_url)

        years = [2018, 2019, 2020, 2021, 2022, 2023]
        url_template_balance = 'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_{year}_{half}.csv'
        url_template_subregion = 'https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_SUBREGION_{year}_{half}.csv'

        process_and_load_data(years, url_template_balance, 'eia930_balance_raw', engine, 'Data Date')
        process_and_load_data(years, url_template_subregion, 'eia930_subregion_raw', engine, 'Data Date')

        t1 = time()
        print(str(datetime.timedelta(seconds=round(t1 - t0))))

    finally:
        print('dn')
        # log_file.close()

if __name__ == "__main__":
    main()


Columns in https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2018_Jan_Jun.csv: ['Balancing Authority', 'Data Date', 'Hour Number', 'Local Time at End of Hour', 'UTC Time at End of Hour', 'Demand Forecast (MW)', 'Demand (MW)', 'Net Generation (MW)', 'Total Interchange (MW)', 'Sum(Valid DIBAs) (MW)', 'Demand (MW) (Imputed)', 'Net Generation (MW) (Imputed)', 'Total Interchange (MW) (Imputed)', 'Demand (MW) (Adjusted)', 'Net Generation (MW) (Adjusted)', 'Total Interchange (MW) (Adjusted)', 'Region']
Columns in https://www.eia.gov/electricity/gridmonitor/sixMonthFiles/EIA930_BALANCE_2018_Jul_Dec.csv: ['Balancing Authority', 'Data Date', 'Hour Number', 'Local Time at End of Hour', 'UTC Time at End of Hour', 'Demand Forecast (MW)', 'Demand (MW)', 'Net Generation (MW)', 'Total Interchange (MW)', 'Sum(Valid DIBAs) (MW)', 'Demand (MW) (Imputed)', 'Net Generation (MW) (Imputed)', 'Total Interchange (MW) (Imputed)', 'Demand (MW) (Adjusted)', 'Net Generation (MW) (Adjusted)',